In [1]:
%load_ext autoreload
%autoreload 2
from main import init_model, training_loop

tokenizer, model = init_model()

/Users/brianfitzgerald/Documents/GitHub/minRL/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-27 12:56:58.848 | INFO     | main:init_model:34 - Loading model Qwen/Qwen2.5-3B-Instruct
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.84s/it]
2025-04-27 12:57:03.687 | INFO     | main:init_model:37 - Model loaded.
2025-04-27 12:57:08.987 | INFO     | main:init_model:40 - Using device mps


In [2]:
model.device

device(type='mps', index=0)

In [3]:
import pandas as pd
df = pd.read_json("connections_prompts.jsonl", lines=True)

In [4]:
from dataset import create_connections_datasets
from main import get_available_device
train_dataset, _ = create_connections_datasets(tokenizer)
device = get_available_device()

In [6]:
training_loop(model, tokenizer, train_dataset, device)

2025-04-27 12:57:16.017 | INFO     | main:training_loop:58 - Starting training loop
2025-04-27 12:57:16.020 | INFO     | main:training_loop:74 - Training loop initialized
2025-04-27 12:57:16.076 | INFO     | main:training_loop:77 - Starting rollout for step 1
2025-04-27 12:57:16.077 | INFO     | grpo:rollout:39 - Generating responses for 1 prompts, max_gen_len=100
2025-04-27 12:57:30.797 | INFO     | grpo:rollout:65 - Generated token ids: 100
2025-04-27 12:57:30.797 | INFO     | grpo:rollout:71 - Generated token ids after removing padding: 100
2025-04-27 12:57:30.797 | INFO     | grpo:rollout:76 - Generated text: acea
<reasoning>
The first group can be formed by words related to transportation or tools: pole, rod, staff, stick, conductor, station, track. These words all relate to equipment used in moving people or vehicles.

The second group contains animals: fox, ibex, lynx, oryx. These words all refer to different species of wild mammals.

The third group consists of words related to

TypeError: reward_function() got an unexpected keyword argument 'answer'